## Create datasets for the Content-based Filter

This notebook builds the data we will use for creating our content based model. We'll collect the data via a collection of SQL queries from the publicly avialable Kurier.at dataset in BigQuery.
Kurier.at is an Austrian newsite. The goal of these labs is to recommend an article for a visitor to the site. In this lab we collect the data for training. In the subsequent notebook we train the recommender model. 

This notebook illustrates
* how to pull data from BigQuery table and write to local files
* how to make reproducible train and test splits 

In [1]:
import os
import tensorflow as tf
import numpy as np
import google.datalab.bigquery as bq

PROJECT = 'qwiklabs-gcp-5a88263a558147b5' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'cloud-training-demos-ml' # REPLACE WITH YOUR BUCKET NAME
REGION = 'northamerica-northeast1-a' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%bash
gcloud  config  set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


We will use this helper funciton to write lists containing article ids, categories, and authors for each article in our database to local file. 

In [3]:
def write_list_to_disk(my_list, filename):
  with open(filename, 'w') as f:
    for item in my_list:
        line = "%s\n" % item
        f.write(str(line.encode('utf8')))

### Pull data from BigQuery

The cell below creates a local text file containing all the article ids (i.e. 'content ids') in the dataset. 

Have a look at the original dataset in [BigQuery](https://bigquery.cloud.google.com/welcome/). Then read through the query below and make sure you understand what it is doing. 

In [4]:
sql="""
#standardSQL
SELECT  
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id 
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY
  content_id
"""
content_ids_list = bq.Query(sql).execute().result().to_dataframe()['content_id'].tolist()
write_list_to_disk(content_ids_list, "content_ids.txt")
print("Some sample content IDs {}".format(content_ids_list[:3]))
print("The total number of articles is {}".format(len(content_ids_list)))

Some sample content IDs ['299824032', '299865757', '299918857']
The total number of articles is 15634


In the following cells, you will create a local file which contains a list of article categories and a list of article authors.

**Hint**: For the TODO below, modify the query above changing 'content_id' to the necessary field and changing index=10 
to the appropriate index.  
Refer back to the original dataset, use the `hits.customDimensions.index` 
field to find the correct index.

In [5]:
sql="""
#standardSQL
SELECT  
  (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category 
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY
  category
"""
categories_list = bq.Query(sql).execute().result().to_dataframe()['category'].tolist()
write_list_to_disk(categories_list, "categories.txt")
print(categories_list)

['Lifestyle', 'News', 'Stars & Kultur']


When creating the author list, we'll only use the first author information for each article.  

In [6]:
sql="""
#standardSQL
SELECT
  REGEXP_EXTRACT((SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)), r"^[^,]+")  AS first_author  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  first_author
"""
authors_list = bq.Query(sql).execute().result().to_dataframe()['first_author'].tolist()
write_list_to_disk(authors_list, "authors.txt")
print("Some sample authors {}".format(authors_list[:10]))
print("The total number of authors is {}".format(len(authors_list)))

Some sample authors ['Marlene Patsalidis', 'Yvonne Widler', 'Thomas  Trescher', 'Johanna Hager', 'Elisabeth Spitzer', 'Daniela Wahl', 'Stefan Berndl', 'Heinz Wagner', 'Brigitte Schokarth', 'Mathias Kainz']
The total number of authors is 385


### Create train and test sets.

In this section, we will create the train/test split of our data for training our model. Read through the query and complete the TODO at the bottom.  
Use the concatenated values for visitor id and content id to create a farm fingerprint, taking 90% of the data for the training set. 

In [7]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL AND
      MOD(ABS(FARM_FINGERPRINT(CONCAT(visitor_id, content_id))), 10) < 9
"""

training_set_df = bq.Query(sql).execute().result().to_dataframe()
training_set_df.to_csv('training_set.csv', header=False, index=False, encoding='utf-8')
training_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1000148716229112932,299913368,News,U4-Störung legt Wiener Frühverkehr lahm,Yvonne Widler,574,299931241
1,1000148716229112932,299931241,Stars & Kultur,Regisseur Michael Haneke kritisiert Flüchtling...,None,574,299913879
2,1000360453832106474,299925700,Lifestyle,Nach Tod von Vater: Tochter bekommt jedes Jahr...,Marlene Patsalidis,574,299922662
3,1000360453832106474,299922662,Lifestyle,Australischer Fernsehstar rechtfertigt Sexismu...,Marlene Patsalidis,574,299826775
4,1001846185946874596,299930679,News,Wintereinbruch naht: Erster Schnee im Osten mö...,Daniela Wahl,574,299930679


Repeat the query as above but change outcome of the farm fingerprint hash to collect the remaining 10% of the data for the test set.

In [8]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL AND
      MOD(ABS(FARM_FINGERPRINT(CONCAT(visitor_id, content_id))), 10) = 9
"""

test_set_df = bq.Query(sql).execute().result().to_dataframe()
test_set_df.to_csv('test_set.csv', header=False, index=False, encoding='utf-8')
test_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1001769331926555188,299921761,News,Bitcoin knackt 10.000-Dollar-Marke,Stefan Hofer,574,299912085
1,1002090131595000997,299931241,Stars & Kultur,Regisseur Michael Haneke kritisiert Flüchtling...,None,574,299813480
2,1002090131595000997,299957318,News,27-Jähriger soll betagte Nachbarin vergewaltig...,Yvonne Widler,574,299826767
3,1002109532017576768,299827911,News,"""Vulkanausbrüche sind normal""",Michaela Reibenwein,574,299965853
4,1002109532017576768,299965853,News,ÖVP und FPÖ dementieren geplante Kassenreduktion,Stefan Berndl,574,299972248


Let's have a look at the two csv files we just created containing the training and test set. We'll also do a line count of both files to confirm that we have achieved an approximate 90/10 train/test split.  
In the next notebook, **Content Based Filtering** we will build a model to recommend an article given information about the current article being read, such as the category, title, author, and publish date. 

In [9]:
%bash
wc -l *_set.csv

   25599 test_set.csv
  232308 training_set.csv
  257907 total


In [10]:
!head *_set.csv

==> test_set.csv <==
1001769331926555188,299921761,News,Bitcoin knackt 10.000-Dollar-Marke,Stefan Hofer,574,299912085
1002090131595000997,299931241,Stars & Kultur,Regisseur Michael Haneke kritisiert Flüchtlingspolitik,,574,299813480
1002090131595000997,299957318,News,27-Jähriger soll betagte Nachbarin vergewaltigt haben,Yvonne Widler,574,299826767
1002109532017576768,299827911,News,"""Vulkanausbrüche sind normal""",Michaela Reibenwein,574,299965853
1002109532017576768,299965853,News,ÖVP und FPÖ dementieren geplante Kassenreduktion,Stefan Berndl,574,299972248
1002109532017576768,299972248,News,"Bildungsexperte Salcher: ""Lehrer nach Leistung bezahlen""",Bernhard Gaul,574,299816215
1007262522112599143,299804319,Stars & Kultur,Matthias Reim spricht über Absturz & Comeback,Elisabeth Spitzer,574,299914459
1008813151291625406,299925700,Lifestyle,Nach Tod von Vater: Tochter bekommt jedes Jahr Blumen,Marlene Patsalidis,574,299170525
1012012064444982688,299824032,Stars & Kultur,YouTube: Schwere